In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm, trange

import os
import sys
import random
import pickle
from functools import reduce
from typing import Any, List, Dict, Tuple, Callable, Iterable, Optional
from dataclasses import dataclass

In [ ]:
if torch.cuda.is_available():
    dev = torch.device('cuda')
else:
    dev = torch.device('cpu')
dev

In [ ]:
seed = int(np.pi * 100_000_000)
torch.random.manual_seed(seed + 1)
np.random.seed(seed + 2)
random.seed(seed + 3)

In [ ]:
skewed_data_dir = 'data/skewed'

In [ ]:
z_syst = torch.tensor(np.sort(np.concatenate([
    np.arange(88, 97) / 100,
    np.arange(970, 1030) / 1000,
    np.arange(103, 107) / 100,
    np.arange(1070, 1130) / 1000,
    np.arange(113, 115) / 100,
], axis=-1)))
z_syst_up = 1.1 
z_syst_down = 0.8 
z_nominal = 1.0

z_syst_train = torch.tensor([
    0.7,  0.74, 0.78, 0.8,  0.84, 0.88, 0.9,  0.92,
    0.94, 0.96, 0.98, 0.99, 1.0,  1.01, 1.02, 1.04,
    1.06, 1.08, 1.09, 1.10, 1.11, 1.12, 1.13, 1.14,
])

In [ ]:
@dataclass
class Dataset:
    x: torch.Tensor
    y: torch.Tensor
    z: torch.Tensor
    weights: torch.Tensor
        
    def __getitem__(self, index) -> 'Dataset':
        return Dataset(
            x=self.x[index],
            y=self.y[index],
            z=self.z[index],
            weights=self.weights[index],
        )
        
    def __setitem__(self, index, item: 'Dataset') -> None:
        self.x[index] = item.x
        self.y[index] = item.y
        self.z[index] = item.z
        self.weights[index] = item.weights
        
    def __len__(self) -> int:
        return len(self.x)
    
    def sizeof(self) -> int:
        return sum(
            reduce(lambda a, b: a * b, tensor.shape) * tensor.element_size()
            for tensor in [
                self.x,
                self.y,
                self.z,
                self.weights,
            ]
        )
    
    def to(self, device) -> 'Dataset':
        return Dataset(
            x=self.x.to(device),
            y=self.y.to(device),
            z=self.z.to(device),
            weights=self.weights.to(device),
        )

In [ ]:
def to_tensor(x, device=None) -> torch.Tensor:
    if device is None:
        device = dev
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return torch.tensor(x.values.astype(np.float32)).to(device)
    if isinstance(x, (np.ndarray, list)):
        return torch.tensor(x).to(device)
    raise TypeError(f'Unknown type {type(x)}')

In [ ]:
def make_dataset(x, y, z, weights) -> Dataset:
    return Dataset(
        x=to_tensor(x, device='cpu'),
        y=to_tensor(y, device='cpu'),
        z=to_tensor(z, device='cpu'),
        weights=to_tensor(weights, device='cpu'),
    )

In [ ]:
def load_training_data_for(z: float) -> Tuple[Dataset, Dataset]:
    path = os.path.join(skewed_data_dir, f'HiggsML_TES_{round(z, 2)}.h5')
    # Read and shuffle.
    df = pd.read_hdf(path, 'data_syst').sample(frac=1).reset_index()
    
    target = df['Label'] == 'b'
    weights = df['Weight']
    z = df['Z']
    assert (z == z[0]).all()
    indices = df['index']
    df.drop(['Label', 'Z', 'Weight', 'index', 'KaggleSet'], axis=1, inplace=True)
    
    train_indices = indices % 2 == 0
    train_set = make_dataset(
        x=df[train_indices],
        y=target[train_indices],
        z=z[train_indices],
        weights=weights[train_indices],
    )
    
    test_indices = ~train_indices
    test_set = make_dataset(
        x=df[test_indices],
        y=target[test_indices],
        z=z[test_indices],
        weights=weights[test_indices],
    )
    
    scale_up = 1.0
    class_weights = (
        weights[target == 0].sum(),
        weights[target == 1].sum(),
    )
    test_class_weights = (
        test_set.weights[test_set.y == 0].sum(),
        test_set.weights[test_set.y == 1].sum(),
    )
    
    for label in (0, 1):
        factor_train = scale_up * max(class_weights) / class_weights[label]
        train_set.weights[train_set.y == label] *= factor_train
        factor_test = class_weights[label] / test_class_weights[label]
        test_set.weights[test_set.y == label] *= factor_test
    
    return train_set, test_set

In [ ]:
def concat_datasets(datasets: List[Dataset]) -> Dataset:
    x = [data.x for data in datasets]
    y = [data.y for data in datasets]
    z = [data.z for data in datasets]
    weights = [data.weights for data in datasets]
    return Dataset(
        x=torch.cat(x),
        y=torch.cat(y),
        z=torch.cat(z),
        weights=torch.cat(weights),
    )

In [ ]:
def shuffle(collection):
    return collection[torch.randperm(len(collection))]

In [ ]:
def load_training_data(
    z_values: List[float],
) -> Tuple[Dataset, Dataset, StandardScaler]:
    train_datasets: List[Dataset] = []
    test_datasets: List[Dataset] = []
    
    total_size = 0
    for z in tqdm(z_values):
        train_dataset, test_dataset = load_training_data_for(float(z))
        train_datasets.append(train_dataset)
        current_size = train_dataset.sizeof() + test_dataset.sizeof()
        total_size += current_size
        print(f'z = {z:.2f}, size = {current_size >> 20}M, total size = {total_size >> 20}M')
        test_datasets.append(test_dataset)
    
    train_cat = concat_datasets(train_datasets)
    test_cat = concat_datasets(test_datasets)
    train_cat = shuffle(train_cat)
    test_cat = shuffle(test_cat)
    
    scaler = StandardScaler()
    train_cat.x = torch.tensor(scaler.fit_transform(train_cat.x), dtype=torch.float32)
    test_cat.x = torch.tensor(scaler.transform(test_cat.x), dtype=torch.float32)
    
    return train_cat, test_cat, scaler

In [ ]:
data_train, data_test, scaler = load_training_data(z_syst_train)

In [ ]:
data_train = data_train.to(dev)
data_test = data_test.to(dev)

In [ ]:
data_train.x.shape

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, num_hidden_layers: int, num_hidden_nodes: int):
        super().__init__()
        input_layer = nn.Sequential(
            nn.Linear(data_train.x.shape[1], num_hidden_nodes),
            nn.ReLU(),
        )
        hidden_layers = [
            nn.Sequential(
                nn.Linear(num_hidden_nodes, num_hidden_nodes),
                nn.ReLU(),
            )
            for _ in range(num_hidden_layers - 1)
        ]
        output_layer = nn.Sequential(
            nn.Linear(num_hidden_nodes, 1),
            nn.Sigmoid(),
        )
        self._layers = nn.Sequential(input_layer, *hidden_layers, output_layer)
        
    def forward(self, x):
        return self._layers(x)

In [ ]:
def train(
    model,
    num_epochs: int,
    batch_size: int,
    dataset_filter: Optional[torch.Tensor] = None,
    Optimizer = torch.optim.RMSprop,
    include_z: bool = False,
):
    dataset = data_train
    if dataset_filter is not None:
        dataset = data_train[dataset_filter]
        
    opt = Optimizer(model.parameters(), weight_decay=0.0001)
    train_dataset = torch.utils.data.TensorDataset(
        dataset.x,
        dataset.y,
        dataset.z,
        dataset.weights,
    )
    loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    for epoch in trange(num_epochs):
        total_loss_value = 0.0
        num_correct = 0
        for x, y, z, weights in tqdm(loader, leave=False):
            opt.zero_grad()
            inputs = (x, z) if include_z else x
            y_pred = model(inputs).flatten()
            loss_value = nn.BCELoss(weight=weights)(y_pred, y)
            loss_value.backward()
            with torch.no_grad():
                total_loss_value += float(loss_value) * len(x)
                num_correct += int(((y_pred > 0.5) == (y > 0.5)).sum())
            opt.step()
        accuracy = num_correct / len(dataset)
        total_loss_value /= len(dataset)
        print(f'Epoch {epoch+1}: loss = {total_loss_value:.5f}, accuracy = {accuracy:.5f}')
    return model

In [ ]:
@dataclass
class Evaluation:
    accuracy: float
    loss: float
        
def evaluate(
    model,
    dataset,
    dataset_filter: Optional[torch.Tensor] = None,
    z = None,
    temp_gpu: bool = True,
    leave_progress_bar: bool = True,
):
    if temp_gpu:
        model.to(dev)
    try:
        with torch.no_grad():
            if dataset_filter is not None:
                dataset = dataset[dataset_filter]
            if z is None:
                test_dataset = torch.utils.data.TensorDataset(
                    dataset.x,
                    dataset.y,
                    dataset.z,
                    dataset.weights,
                )
            else:
                test_dataset = torch.utils.data.TensorDataset(
                    dataset.x,
                    dataset.y,
                    z,
                    dataset.weights,
                )
            loader = torch.utils.data.DataLoader(test_dataset, batch_size=4096, shuffle=True)
            loss_value = 0.0
            num_correct = 0
            for x, y, zvalue, weights in tqdm(loader, leave=leave_progress_bar):
                inputs = x if z is None else (x, zvalue)
                y_pred = model(inputs).flatten()
                loss_value += float(nn.BCELoss(weight=weights)(y_pred, y)) * len(x)
                num_correct += int(((y_pred > 0.5) == (y > 0.5)).sum())

            accuracy = num_correct / len(dataset)
            return Evaluation(accuracy=accuracy, loss=loss_value/len(dataset))
    finally:
        if temp_gpu:
            model.cpu()

## Training

### Baseline model (nominal z)

In [ ]:
baseline_nominal_model = train(
    BaselineModel(num_hidden_layers=10, num_hidden_nodes=512).to(dev),
    num_epochs=200,
    batch_size=2048,
    Optimizer=lambda *a, **k: torch.optim.RMSprop(*a, **k, lr=0.001, alpha=0.9),
    dataset_filter=torch.isclose(data_train.z, torch.tensor(z_nominal, dtype=torch.float32)),
).cpu()

In [ ]:
evaluate(
    baseline_nominal_model,
    data_test,
    torch.isclose(data_test.z, torch.tensor(z_nominal, dtype=torch.float32)),
)

In [ ]:
with open('data/model.baseline-nominal.pt', 'wb') as f:
    torch.save(baseline_nominal_model, f)

### Baseline model (low z)

In [ ]:
baseline_down_model = train(
    BaselineModel(num_hidden_layers=10, num_hidden_nodes=64).to(dev),
    num_epochs=200,
    batch_size=2048,
    Optimizer=lambda *a, **k: torch.optim.RMSprop(*a, **k, lr=0.001, alpha=0.9),
    dataset_filter=torch.isclose(data_train.z, torch.tensor(z_syst_down, dtype=torch.float32)),
).cpu()

In [ ]:
evaluate(
    baseline_down_model,
    data_test,
    torch.isclose(data_test.z, torch.tensor(z_syst_down, dtype=torch.float32)),
)

In [ ]:
with open('data/model.baseline-down.pt', 'wb') as f:
    torch.save(baseline_down_model, f)

### Baseline model (high z)

In [ ]:
baseline_up_model = train(
    BaselineModel(num_hidden_layers=10, num_hidden_nodes=512).to(dev),
    num_epochs=200,
    batch_size=2048,
    Optimizer=lambda *a, **k: torch.optim.RMSprop(*a, **k, lr=0.001, alpha=0.9),
    dataset_filter=torch.isclose(data_train.z, torch.tensor(z_syst_up, dtype=torch.float32)),
).cpu()

In [ ]:
evaluate(
    baseline_up_model,
    data_test,
    torch.isclose(data_test.z, torch.tensor(z_syst_up, dtype=torch.float32)),
)

In [ ]:
with open('data/model.baseline-up.pt', 'wb') as f:
    torch.save(baseline_up_model, f)

### Data augmentation model

In [ ]:
aug_model = train(
    BaselineModel(num_hidden_layers=10, num_hidden_nodes=64).to(dev),
    num_epochs=10,
    batch_size=2048,
    Optimizer=lambda *a, **k: torch.optim.Adam(*a, **k, lr=0.001),
).cpu()

In [ ]:
evaluate(
    aug_model,
    data_test,
)

In [ ]:
with open('data/model.aug.pt', 'wb') as f:
    torch.save(aug_model, f)

### Uncertainty aware model

In [ ]:
class UncertaintyAwareModel(nn.Module):
    def __init__(self, num_hidden_layers: int, num_hidden_nodes: int):
        super().__init__()
        self._layers_a = UncertaintyAwareModel.create_layers(num_hidden_layers, num_hidden_nodes)
        self._layers_b = UncertaintyAwareModel.create_layers(num_hidden_layers, num_hidden_nodes)
    
    @staticmethod
    def create_layers(num_hidden_layers: int, num_hidden_nodes: int):
        input_layer = nn.Sequential(
            nn.Linear(data_train.x.shape[1] + 1, num_hidden_nodes),
            nn.ReLU(),
        )
        hidden_layers = [
            nn.Sequential(
                nn.Linear(num_hidden_nodes, num_hidden_nodes),
                nn.ReLU(),
            )
            for _ in range(num_hidden_layers - 1)
        ]
        output_layer = nn.Sequential(
            nn.Linear(num_hidden_nodes, 1),
            nn.Sigmoid(),
        )
        return nn.Sequential(input_layer, *hidden_layers, output_layer)
    
    def forward(self, inputs):
        x, z = inputs
        z = z.view((len(z), 1))
        input_tensor = torch.cat([x, z], dim=1)
        
        a = self._layers_a(input_tensor)
        b = self._layers_b(input_tensor)
        return torch.where(z < 1, a, b)

In [ ]:
aware_model = train(
    UncertaintyAwareModel(num_hidden_layers=10, num_hidden_nodes=64).to(dev),
    num_epochs=15,
    batch_size=2048,
    include_z=True,
).cpu()

In [ ]:
evaluate(aware_model, data_test, z=data_test.z)

In [ ]:
with open('data/model.aware.pt', 'wb') as f:
    torch.save(aware_model, f)